In [ ]:
!pip install langchain-openai

In [ ]:
import os
import openai

os.environ['OPENAI_API_KEY'] = "YOUR API KEY"

In [ ]:
def perform_arithmetic(first_num: int, second_num: int, operation: str) -> int:
    """Perform arithmetic on two numbers.

    Args:
        first_num: first number in the arithmetic operation. If dividing, this would be the numerator.
        second_num: second number in the arithmetic operation. If dividing, this would be the denominator.
        operation: one of (add, subtract, multiply, divide)
    """

    if operation == 'add':
      return first_num + second_num
    elif operation == 'subtract':
      return first_num - second_num
    elif operation == 'multiply':
      return first_num * second_num
    elif operation == 'divide':
      return first_num / second_num
    else:
      return f"The following operation is not allowed: {operation}"

In [ ]:
from langchain_core.utils.function_calling import convert_to_openai_tool
import json

arithmetic_function = convert_to_openai_tool(perform_arithmetic)
print(json.dumps(arithmetic_function, indent=4))

{
    "type": "function",
    "function": {
        "name": "perform_arithmetic",
        "description": "Perform arithmetic on two numbers.",
        "parameters": {
            "type": "object",
            "properties": {
                "first_num": {
                    "type": "integer",
                    "description": "first number in the arithmetic operation. If dividing, this would be the numerator."
                },
                "second_num": {
                    "type": "integer",
                    "description": "second number in the arithmetic operation. If dividing, this would be the denominator."
                },
                "operation": {
                    "type": "string",
                    "description": "one of (add, subtract, multiply, divide)"
                }
            },
            "required": [
                "first_num",
                "second_num",
                "operation"
            ]
        }
    }
}


In [ ]:
from langchain_openai import ChatOpenAI

reg_model = ChatOpenAI(model="gpt-3.5-turbo")

In [ ]:
reg_model.invoke('what is 910751 * 81301589?')

AIMessage(content='74,223,496,970,639')

In [ ]:
model_with_tool = reg_model.bind(
    tools=[arithmetic_function],
)

In [ ]:
result = model_with_tool.invoke('what is 910751 * 81301589?')
print(result)

content='' additional_kwargs={'tool_calls': [{'id': 'call_lQbyDWr7mAZaag4GRCve14qj', 'function': {'arguments': '{"first_num":910751,"second_num":81301589,"operation":"multiply"}', 'name': 'perform_arithmetic'}, 'type': 'function'}]}


In [ ]:
args = result.additional_kwargs['tool_calls'][0]['function']['arguments']
function_name = result.additional_kwargs['tool_calls'][0]['function']['name']

print("Arguments: ", args)
print("Function Name: ", function_name)

Arguments:  {"first_num":910751,"second_num":81301589,"operation":"multiply"}
Function Name:  perform_arithmetic


In [ ]:
refined_args = json.loads(args)
answer = perform_arithmetic(**refined_args)
print(f"ANSWER: {answer}")

ANSWER: 74045503483339


### Putting it all together into a Chatbot

In [ ]:
from langchain_core.tools import tool

@tool
def perform_arithmetic(first_num: int, second_num: int, operation: str) -> int:
    """Multiply two integers together.

    Args:
        first_num: first number in the arithmetic operation. If dividing, this would be the numerator.
        second_num: second number in the arithmetic operation. If dividing, this would be the denominator.
        operation: one of (add, subtract, multiply, divide)
    """

    if operation == 'add':
      return first_num + second_num
    elif operation == 'subtract':
      return first_num - second_num
    elif operation == 'multiply':
      return first_num * second_num
    elif operation == 'divide':
      return first_num / second_num
    else:
      return f"The following operation is not allowed: {operation}"

tools = [perform_arithmetic]

In [ ]:
!pip install langchainhub langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 816.1/816.1 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 7.4 MB/s eta 0:00:00


In [ ]:
from langchain import hub
from langchain.agents import AgentExecutor, create_openai_tools_agent
from langchain_openai import ChatOpenAI

In [ ]:
prompt = hub.pull("hwchase17/openai-tools-agent")
prompt.pretty_print()

================================ System Message ================================

You are a helpful assistant

============================= Messages Placeholder =============================

{chat_history}

================================ Human Message =================================

{input}

============================= Messages Placeholder =============================

{agent_scratchpad}


In [ ]:
prompt[0].prompt.template = 'Your job is to perform math calculations using the perform_arithmetic function'
prompt[0]

SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='Your job is to perform math calculations using the perform_arithmetic function'))

In [ ]:
prompt[1]

MessagesPlaceholder(variable_name='chat_history', optional=True)

In [ ]:
prompt[2]

HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}'))

In [ ]:
prompt[3]

MessagesPlaceholder(variable_name='agent_scratchpad')

In [ ]:
model = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0)

# Construct the OpenAI Tools agent
agent = create_openai_tools_agent(model, tools, prompt)

In [ ]:
# Create an agent executor by passing in the agent and tools
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [ ]:
agent_executor.invoke(
    {
        "input": "what is ((910751 * 81301589) / 355) - 734872810242?"
    }
)



> Entering new AgentExecutor chain...

Invoking: `perform_arithmetic` with `{'first_num': 910751, 'second_num': 81301589, 'operation': 'multiply'}`


74045503483339
Invoking: `perform_arithmetic` with `{'first_num': 74045503483339, 'second_num': 355, 'operation': 'divide'}`


208578883051.65915
Invoking: `perform_arithmetic` with `{'first_num': 208578883051, 'second_num': 734872810242, 'operation': 'subtract'}`


-526293927191The result of the expression ((910751 * 81301589) / 355) - 734872810242 is -526,293,927,191.

> Finished chain.


{'input': 'what is ((910751 * 81301589) / 355) - 734872810242?',
 'output': 'The result of the expression ((910751 * 81301589) / 355) - 734872810242 is -526,293,927,191.'}

### Economics Chatbot

In [ ]:
import requests
import json

@tool
def pull_economic_data(economic_indicator, start_year, end_year):
    """
    Pull historical economic data.

    Args:
        economic_indicator: Which economic indicator you would like to pull historical data on. This can be one of: CPI, PPI, or unemployment_rate
        start_year: the start year of the date range over which you want to pull economic data
        end_year: the end year of the date range over which you want to pull economic data
    """

    # See this for series ids: https://data.bls.gov/cgi-bin/surveymost?bls
    series_map = {'CPI': 'CUUR0000SA0',
                  'PPI': 'WPUFD4',
                  'unemployment_rate': 'LNS14000000'}

    resp = requests.post(
        url='https://api.bls.gov/publicAPI/v1/timeseries/data/',
        data=json.dumps({'seriesid': [series_map.get(economic_indicator)], 'startyear': start_year, 'endyear': end_year}),
        headers={'Content-type': 'application/json'}
    )

    data = json.loads(resp.text)

    try:
      # refine it
      refined_data = []
      for series in data['Results']['series']:
        for item in series['data']:
          refined_data.append(item)

    except:
      refined_data = []

    return refined_data


# params = {'economic_indicator': 'unemployment_rate', 'start_year': 2023, 'end_year': 2024}
# data = pull_economic_data(economic_indicator=params['economic_indicator'], start_year='2015', end_year='2024')

# for item in data[:5]:
#   print(item)

In [ ]:
from datetime import date

prompt[0].prompt.template = f'Your job is to answer users economic questions by referencing economic data. Note that todays date is {str(date.today())}'
prompt[0]

SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='Your job is to answer users economic questions by referencing economic data. Note that todays date is 2024-02-22'))

In [ ]:
model = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0)
agent = create_openai_tools_agent(model, [pull_economic_data], prompt)
agent_executor = AgentExecutor(agent=agent, tools=[pull_economic_data], verbose=True)

In [ ]:
resp = agent_executor.invoke(
    {
        "input": "Pull umemployment rate data for 2023 and 2024"
    }
)



> Entering new AgentExecutor chain...

Invoking: `pull_economic_data` with `{'economic_indicator': 'unemployment_rate', 'start_year': 2023, 'end_year': 2024}`


[{'year': '2024', 'period': 'M01', 'periodName': 'January', 'latest': 'true', 'value': '3.7', 'footnotes': [{}]}, {'year': '2023', 'period': 'M12', 'periodName': 'December', 'value': '3.7', 'footnotes': [{}]}, {'year': '2023', 'period': 'M11', 'periodName': 'November', 'value': '3.7', 'footnotes': [{}]}, {'year': '2023', 'period': 'M10', 'periodName': 'October', 'value': '3.8', 'footnotes': [{}]}, {'year': '2023', 'period': 'M09', 'periodName': 'September', 'value': '3.8', 'footnotes': [{}]}, {'year': '2023', 'period': 'M08', 'periodName': 'August', 'value': '3.8', 'footnotes': [{}]}, {'year': '2023', 'period': 'M07', 'periodName': 'July', 'value': '3.5', 'footnotes': [{}]}, {'year': '2023', 'period': 'M06', 'periodName': 'June', 'value': '3.6', 'footnotes': [{}]}, {'year': '2023', 'period': 'M05', 'periodName': 'May', 'value

In [ ]:
print(resp['output'])

The unemployment rate in the US for May 2023 was 3.7%.
